In [23]:
import os
import pickle
import itertools
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsforecast.models import MSTL
from sklearn.metrics import mean_pinball_loss

os.chdir("C:/2023_11-PTSFC")
import model_train as model_train
import data_prepro as data_prepro

### Params

In [14]:
os.environ["LOKY_MAX_CPU_COUNT"] = "1"  # Replace "4" with the desired number of cores

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
fcast_hor = [36, 40, 44, 60, 64, 68] # in hours

### Data Prep

In [15]:
# = = = = = = = = = = = = = 
# get data
# df_energy = data_prepro.get_energy_data_today(to_date=t_wednesday.strftime('%Y%m%d'))

# Read data from file with specified data types
df_energy = pd.read_csv("data/2015-01-01_2024-02-21_energy.csv", index_col=0, parse_dates=[0])
df_energy['timestamp_CET'] = pd.to_datetime(df_energy['timestamp_CET'], utc=True).dt.tz_convert('CET')
print(df_energy.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 80136 entries, 2014-12-31 23:00:00+00:00 to 2024-02-21 22:00:00+00:00
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   timestamp_CET  80136 non-null  datetime64[ns, CET]
 1   gesamt         80136 non-null  float64            
dtypes: datetime64[ns, CET](1), float64(1)
memory usage: 1.8 MB
None


In [24]:
def preprocess_data(df_energy, start_date):
    
    df_energy_small = df_energy.loc[(df_energy['timestamp_CET'] > start_date)].copy()
    
    df_energy_dummy = data_prepro.create_dummy_df(df_energy_small, hour_method='simple', holiday_method='separate')
    df_energy_fturs = data_prepro.create_features_df(df_energy_small, holiday_method='separate')

    X_train_fturs = df_energy_fturs.drop(['gesamt', 'timestamp_CET'], axis=1)
    y_train_fturs = df_energy_fturs['gesamt']

    X_train_dummy = df_energy_dummy.drop(['gesamt', 'timestamp_CET'], axis=1)
    y_train_dummy = df_energy_dummy['gesamt']
    
    return X_train_dummy, y_train_dummy, X_train_fturs, y_train_fturs

def generate_param_grids(params):
    
        param_values = list(itertools.product(*params.values()))
        param_names = list(params.keys())

        param_grids = []

        for values in param_values:
            param_dict = dict(zip(param_names, values))
            param_grids.append(param_dict)

        return param_grids

lgbm_params = {
    'max_depth': [4, 10],
    'num_leaves': [5, 15, 20],
    'learning_rate': [0.1, 0.3],
    'n_estimators': [100, 200],
    'boosting_type': ['gbdt'],
    'verbose': [-1]
}

xgb_params = {
    'objective': ['reg:quantileerror'],
    'eval_metric': ['quantile'],
    'booster': ['gbtree'],
    'max_depth': [4, 10],
    'learning_rate': [0.2],
    'n_estimators': [100, 200],
}

all_lgbm_params = generate_param_grids(lgbm_params)
all_xgb_params = generate_param_grids(xgb_params)

In [25]:
# = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
# import trained models
path = "custom_models"

def load_pickled_object(path, filename):
    with open(f"{path}/{filename}", "rb") as handle:
        return pickle.load(handle)
    
quant_reg_models = {}
quant_reg_models['quant_reg_2015_sk'] = load_pickled_object(path, "20150101_20231031_quant_reg_sk.pickle")
quant_reg_models['quant_reg_2018_sk'] = load_pickled_object(path, "20180101_20231031_quant_reg_sk.pickle")
quant_reg_models['quant_reg_2015_sm'] = load_pickled_object(path, "20150101_20231031_quant_reg_sm.pickle")
quant_reg_models['quant_reg_2018_sm'] = load_pickled_object(path, "20180101_20231031_quant_reg_sm.pickle")

grad_boost_models = {}
grad_boost_models['grad_boost_2015_fturs'] = load_pickled_object(path, "20150101_20231031_grad_boost_fturs.pickle")
grad_boost_models['grad_boost_2018_fturs'] = load_pickled_object(path, "20180101_20231031_grad_boost_fturs.pickle")
grad_boost_models['grad_boost_2015_dummy'] = load_pickled_object(path, "20150101_20231031_grad_boost_dummy.pickle")
grad_boost_models['grad_boost_2018_dummy'] = load_pickled_object(path, "20180101_20231031_grad_boost_dummy.pickle")

c:\Users\ytl_c\miniconda3\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DummyRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ytl_c\miniconda3\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ytl_c\miniconda3\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.3.0 when using version 1.3.2. This m

In [27]:
cwd = os.getcwd()
folder_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = os.path.join(cwd, folder_name)

if not os.path.exists(directory):
    os.makedirs(directory)
os.chdir(directory)

# Define the start and end dates
start_date = pd.Timestamp('2023-11-15')
end_date = pd.Timestamp('2024-02-14')

# Generate a list of weekly dates in UTC
fcast_dates_cet = pd.date_range(start=start_date, end=end_date, freq='W-WED').tz_localize('CET').strftime('%Y-%m-%d').tolist()
dict_all_fcasts = {}
dict_all_evals = {}

# Iterate over the forecast dates
for fcast_date in fcast_dates_cet[:]:

    print('= '*30)
    print(f"Forecasting for week starting from {fcast_date} ...")
    
    dict_weekly_fcasts = {}
    dict_weekly_models = {}

    # = = = = = = = = = = = = = 
    # generate prediction timestamps based on t0 = following thursday 00:00
    # = = = = = = = = = = = = = 

    # Calculate the Thursday and Wednesday of the week
    t_wednesday = pd.Timestamp(fcast_date).replace(hour=0, minute=0, second=0, microsecond=0).tz_localize('CET')
    t_thursday = t_wednesday + pd.Timedelta(days=1)

    # Generate required submission timestamps
    subm_timestamps = [(t_thursday + pd.Timedelta(hours=fcast)) for fcast in fcast_hor]
    print(f"Submission timestamps = {subm_timestamps[0]} to {subm_timestamps[-1]}")
    
    # Create df with Information at fcast start date
    t_thursday_str = t_thursday.strftime('%Y-%m-%d')
    df_energy_current = df_energy.loc[df_energy['timestamp_CET'] <= t_thursday_str].copy()
    print('= '*30)

    # = = = = = = = = = = = = = 
    # Data Prep for All Methods
    # = = = = = = = = = = = = = 
    
    # create fcast index for next 68 hours
    fcast_timestamp_CET = pd.date_range(start=t_thursday, periods=68+1, freq='H')
    fcast_timestamp_UTC = fcast_timestamp_CET.tz_convert('UTC')

    # create df with fcast timestamps as INPUT for model
    df_temp = pd.DataFrame(index=fcast_timestamp_UTC)
    df_temp['timestamp_CET'] = fcast_timestamp_CET
    df_fcast_dummy = data_prepro.create_dummy_df(df_temp, hour_method='simple', holiday_method='separate')

    # = = = = = = = = = = = = = 
    # Simple Benchmark
    # = = = = = = = = = = = = = 
    
    # create new dataframe with relevant info for benchmark
    df_energy_benchmark = df_energy_current.copy()
    df_energy_benchmark["month"] = df_energy_benchmark['timestamp_CET'].dt.month
    df_energy_benchmark["weekday"] = df_energy_benchmark['timestamp_CET'].dt.weekday # Monday=0, Sunday=6
    df_energy_benchmark["weeknum"] = df_energy_benchmark['timestamp_CET'].dt.isocalendar().week

    last_t = 150 # if there are more matches only take the most recent
    pred_baseline = np.zeros((3,len(fcast_timestamp_CET),5)) # 3 condition types, 5 quantiles

    for i,d in enumerate(fcast_timestamp_CET):
            
        weekday = d.weekday()
        hour = d.hour
        weeknum = d.week
        
        # basic condition that the weekday and hour match
        basic_cond = (df_energy_benchmark.weekday == weekday) & (df_energy_benchmark.index.time == d.time())
        
        # AND the weeknum is within +/- 2 weeks of the target
        cond1 = (df_energy_benchmark['weeknum'].between(weeknum-2, weeknum+2)) 
        # AND the month also matches
        cond2 = (df_energy_benchmark.index.month == d.month)
        # AND the month is within +/- 1 months of the target
        cond3 = (df_energy_benchmark['month'].between(d.month-1, d.month+1))

        cond_list = [cond1, cond2, cond3]

        for cond_idx, cond in enumerate(cond_list):
            cond = basic_cond & cond
            match_df = df_energy_benchmark[cond]
            # print(f"condition {cond_idx} ... {len(match_df)} matches found")
            pred_baseline[cond_idx, i, :] = np.quantile(match_df.iloc[-last_t:]["gesamt"], q=quantiles) # method='linear'

    methods = ['bench_pm_2weeks', 'bench_same_month', 'bench_pm_1month']
    for m_idx, method in enumerate(methods):

        print(f"method = {method}")
        # create empty df
        df_benchmark = pd.DataFrame(index=fcast_timestamp_UTC, columns=[f"q {q:.3f}" for q in quantiles])
        df_benchmark.loc[:,:] = pred_baseline[m_idx,:,:]

        # make sure all cols are float
        df_benchmark = df_benchmark.astype(float)
        # add CET col
        df_benchmark['timestamp_CET'] = df_benchmark.index.tz_convert('CET')
        # reorder cols
        df_benchmark = df_benchmark[['timestamp_CET', 'q 0.025', 'q 0.250', 'q 0.500', 'q 0.750', 'q 0.975']]
        # save to dict
        dict_weekly_fcasts[method] = df_benchmark

    # = = = = = = = = = = = = = 
    # MSTL
    # = = = = = = = = = = = = = 

    mstl_train_horizon = 0.5 # in years

    for mstl_train_horizon in [1, 0.5]:
        
        method = f"mstl_{mstl_train_horizon}"
        print(f"method = {method}")

        df_mstl_train = df_energy_current.iloc[-int(mstl_train_horizon * 365 * 24):].copy()
        mstl_model = MSTL(season_length=[24, 24 * 7]).fit(df_mstl_train["gesamt"])

        n_steps = df_benchmark.shape[0]

        y_hat_dict = mstl_model.predict(h=n_steps, level=[50, 95])
        y_hat_df = pd.DataFrame(y_hat_dict)
        y_hat_df["timestamp_CET"] = pd.date_range(start=t_thursday, periods=len(y_hat_df), freq="H")

        # rename columns
        y_hat_df = y_hat_df.rename(
            columns={
                "mean": "q 0.500",
                "lo-50": "q 0.250",
                "hi-50": "q 0.750",
                "lo-95": "q 0.025",
                "hi-95": "q 0.975",
            }
        )

        # rearrange cols
        y_hat_df = y_hat_df[["timestamp_CET", "q 0.025", "q 0.250", "q 0.500", "q 0.750", "q 0.975"]]

        df_mstl_fcast = y_hat_df
        df_mstl_fcast.index = fcast_timestamp_UTC

        dict_weekly_fcasts[method] = df_mstl_fcast
    
    # = = = = = = = = = = = = = 
    # Quantile Reg
    # = = = = = = = = = = = = = 
    
    for method, all_models_quant_reg in quant_reg_models.items():
        
        print(f"method = {method}")

        # create empty OUTPUT df with columns = quantiles
        df_quant_reg_direct_fcast = pd.DataFrame(index=df_fcast_dummy.index)
        df_quant_reg_direct_fcast['timestamp_CET'] = fcast_timestamp_CET

        # Prediction for Quantile Regression
        for name, model in sorted(all_models_quant_reg.items()):
            pred = model.predict(df_fcast_dummy.drop('timestamp_CET', axis=1))
            df_quant_reg_direct_fcast[name] = pred

        dict_weekly_fcasts[method] = df_quant_reg_direct_fcast

    # = = = = = = = = = = = = = 
    # Grad Boosting
    # = = = = = = = = = = = = = 

    for method, all_models_grad_boost in grad_boost_models.items():

        print(f"method = {method}")

        if "fturs" in method:
            df_fcast_temp = data_prepro.create_features_df(df_temp, holiday_method='separate')
            
        elif "dummy" in method:
            df_fcast_temp = data_prepro.create_dummy_df(df_temp, hour_method='simple', holiday_method='separate')

        # create empty OUTPUT df with columns = quantiles
        df_grad_boost_direct_fcast = pd.DataFrame(index=df_fcast_temp.index)
        df_grad_boost_direct_fcast['timestamp_CET'] = fcast_timestamp_CET

        # Prediction for Quantile Regression
        for name, model in sorted(all_models_grad_boost.items()):
            pred = model.predict(df_fcast_temp.drop('timestamp_CET', axis=1))
            df_grad_boost_direct_fcast[name] = pred

        dict_weekly_fcasts[method] = df_grad_boost_direct_fcast
        
    # = = = = = = = = = = = = = 
    # LightGBM
    # = = = = = = = = = = = = = 
    
    yr = 2020
    start_date = f"{yr}-01-01"
    X_train_dummy, y_train_dummy, X_train_fturs, y_train_fturs = preprocess_data(df_energy_current, start_date)

    methods = [f"lightgbm_dummy_{yr}_{i}" for i in range(len(all_lgbm_params))]
    
    for method_idx, method in enumerate(methods):

        print(f"method = {method}")
        params = all_lgbm_params[method_idx]

        if "fturs" in method:
            all_models = model_train.fit_lightgbm(X_train_fturs, y_train_fturs, quantiles, params)
            df_fcast_temp = data_prepro.create_features_df(df_temp, holiday_method='separate')
            
        elif "dummy" in method:
            all_models = model_train.fit_lightgbm(X_train_dummy, y_train_dummy, quantiles, params)
            df_fcast_temp = data_prepro.create_dummy_df(df_temp, hour_method='simple', holiday_method='separate')

        # create empty OUTPUT df with columns = quantiles
        df_direct_fcast = pd.DataFrame(index=df_fcast_temp.index)
        df_direct_fcast['timestamp_CET'] = fcast_timestamp_CET

        # Prediction
        for name, model in sorted(all_models.items()):
            pred = model.predict(df_fcast_temp.drop('timestamp_CET', axis=1))
            df_direct_fcast[name] = pred

        dict_weekly_fcasts[method] = df_direct_fcast
        
    # = = = = = = = = = = = = = 
    # XGBoost model
    # = = = = = = = = = = = = = 

    X_train_dummy, y_train_dummy, X_train_fturs, y_train_fturs = preprocess_data(df_energy_current, start_date)
    
    df_fcast_dummy = data_prepro.create_dummy_df(df_temp, hour_method='simple', holiday_method='separate')
    
    methods = [f"xgboost_dummy_{yr}_{i}" for i in range(len(all_xgb_params))]
    
    for method_idx, method in enumerate(methods):

        print(f"method = {method}")
        params = all_xgb_params[method_idx]
    
        all_models = model_train.fit_xgboost(X_train_dummy, y_train_dummy, quantiles, params)
        dict_weekly_models[method] = all_models

        # create empty OUTPUT df with columns = quantiles
        df_direct_fcast = pd.DataFrame(index=df_fcast_dummy.index)
        df_direct_fcast['timestamp_CET'] = fcast_timestamp_CET

        # Prediction
        for name, model in sorted(all_models.items()):
            pred = model.predict(df_fcast_dummy.drop('timestamp_CET', axis=1))
            df_direct_fcast[name] = pred

        dict_weekly_fcasts[method] = df_direct_fcast
    
    # = = = = = = = = = = = = = 
    # Evaluation based on submission timestamps
    # = = = = = = = = = = = = = 

    # get actual values at every submission timestamp
    df_energy_eval = df_energy.loc[df_energy['timestamp_CET'].isin(subm_timestamps)].copy()
    evaluation_results = {}
    
    for model_name, forecast_df in dict_weekly_fcasts.items():

        # Initialize an empty DataFrame to store quantile scores
        quantile_scores = pd.DataFrame(index=subm_timestamps, columns=[f"q {q:.3f}" for q in quantiles])
        # take subset of fcast df at submission timestamps
        forecast_df = forecast_df.loc[forecast_df['timestamp_CET'].isin(subm_timestamps)].copy()

        # Iterate over each submission timestamp
        for q_idx, q in enumerate(quantiles):

            qscore = mean_pinball_loss(alpha=q, 
                                       y_true=df_energy_eval['gesamt'].values, 
                                       y_pred=forecast_df.iloc[:,q_idx+1].values) # skip timestamp_CET col
            
            quantile_scores.iloc[:,q_idx] = qscore / 1000
        
        # Store the quantile scores for the model
        evaluation_results[model_name] = quantile_scores
    
    # Calculate mean scores for each quantile over time
    mean_scores = {}
    for model_name, quantile_scores in evaluation_results.items():
        mean_scores[model_name] = quantile_scores.mean()
    
    # df = pd.DataFrame(mean_scores).T
    # filtered_df = df[df.index.str.contains('lightgbm')]
    # display(filtered_df.style.highlight_min(color='lightgreen', axis=0))

    # df = pd.DataFrame(mean_scores).T
    # filtered_df = df[df.index.str.contains('xgboost')]
    # display(filtered_df.style.highlight_min(color='lightgreen', axis=0))

    # calculate mean scores over all quantiles
    mean_scores_df = pd.DataFrame(mean_scores)
    
    print('- '*15)
    print('scores:')
    print(mean_scores_df.mean(axis=0).sort_values(ascending=True))
        
    # = = = = = = = = = = = = = 
    # Save all fcasts & trained models for the week
    # = = = = = = = = = = = = = 
    
    dict_all_fcasts[fcast_date] = dict_weekly_fcasts
    dict_all_evals[fcast_date] = evaluation_results
    
    with open(f'{fcast_date}_models.pickle', 'wb') as handle:
        pickle.dump(dict_weekly_models, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f'eval.pickle', 'wb') as handle:
    pickle.dump(dict_all_evals, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'fcasts.pickle', 'wb') as handle:
    pickle.dump(dict_all_fcasts, handle, protocol=pickle.HIGHEST_PROTOCOL)

os.chdir(cwd)

= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
Forecasting for week starting from 2023-11-15 ...
Submission timestamps = 2023-11-17 12:00:00+01:00 to 2023-11-18 20:00:00+01:00
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
method = bench_pm_2weeks
method = bench_same_month
method = bench_pm_1month
method = mstl_1
method = mstl_0.5
method = quant_reg_2015_sk
method = quant_reg_2018_sk
method = quant_reg_2015_sm
method = quant_reg_2018_sm
method = grad_boost_2015_fturs
method = grad_boost_2018_fturs
method = grad_boost_2015_dummy
method = grad_boost_2018_dummy
method = lightgbm_dummy_2020_0
method = lightgbm_dummy_2020_1
method = lightgbm_dummy_2020_2
method = lightgbm_dummy_2020_3
method = lightgbm_dummy_2020_4
method = lightgbm_dummy_2020_5
method = lightgbm_dummy_2020_6
method = lightgbm_dummy_2020_7
method = lightgbm_dummy_2020_8
method = lightgbm_dummy_2020_9
method = lightgbm_dummy_2020_10
method = lightgbm_dummy_2020_11
method = lightgbm_dummy_2020_12
m